In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import AdamW
from sklearn.metrics import classification_report, cohen_kappa_score
import numpy as np



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
IMG_SIZE = 380
BATCH_SIZE = 16   # safe for B4
EPOCHS = 10
NUM_CLASSES = len(tf.io.gfile.listdir(
    '/content/drive/MyDrive/train_5000_per_class'
))

train_dir = '/content/drive/MyDrive/train_5000_per_class'
val_dir   = '/content/drive/MyDrive/val_5000_per_class'
test_dir  = '/content/drive/MyDrive/test_5000_per_class'


In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

val_gen = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

test_gen = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)


Found 25000 images belonging to 5 classes.
Found 12332 images belonging to 5 classes.
Found 12305 images belonging to 5 classes.


In [ ]:
base_model = EfficientNetB4(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

base_model.trainable = False  # Phase-1


71686520/71686520 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)

model = models.Model(inputs, outputs)


In [ ]:
model.compile(
    optimizer=AdamW(learning_rate=1e-4, weight_decay=1e-4),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)


In [ ]:
callbacks = [
    EarlyStopping(patience=8, restore_best_weights=True),
    ReduceLROnPlateau(patience=4, factor=0.3),
    ModelCheckpoint(
        'best_effnetb4.h5',
        monitor='val_accuracy',
        save_best_only=True
    )
]


In [ ]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    callbacks=callbacks
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.2054 - loss: 1.7840

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11792s 8s/step - accuracy: 0.2054 - loss: 1.7839 - val_accuracy: 0.3604 - val_loss: 1.5985 - learning_rate: 1.0000e-04
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 812s 519ms/step - accuracy: 0.2047 - loss: 1.7113 - val_accuracy: 0.2274 - val_loss: 1.6129 - learning_rate: 1.0000e-04
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 811s 519ms/step - accuracy: 0.2071 - loss: 1.6823 - val_accuracy: 0.1743 - val_loss: 1.6162 - learning_rate: 1.0000e-04
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 810s 518ms/step - accuracy: 0.2075 - loss: 1.6628 - val_accuracy: 0.2068 - val_loss: 1.6074 - learning_rate: 1.0000e-04
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 806s 516ms/step - accuracy: 0.2071 - loss: 1.6498 - val_accuracy: 0.2804 - val_loss: 1.6043 - learning_rate: 1.0000e-04
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 809s 518ms/step - accuracy: 0.2137 - loss: 1.6247 - val_accuracy: 0.2135 - val_loss: 1.6053 - learning_rate: 3.0000e-05
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━

In [ ]:
base_model.trainable = True

for layer in base_model.layers[:-30]:
    layer.trainable = False


In [ ]:
model.compile(
    optimizer=AdamW(learning_rate=1e-5, weight_decay=1e-4),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.05),
    metrics=['accuracy']
)


In [ ]:
fine_tune_history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    callbacks=callbacks
)


Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 913s 553ms/step - accuracy: 0.2064 - loss: 2.0448 - val_accuracy: 0.1530 - val_loss: 1.6959 - learning_rate: 1.0000e-05
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 813s 520ms/step - accuracy: 0.2069 - loss: 1.9083 - val_accuracy: 0.1010 - val_loss: 1.6560 - learning_rate: 1.0000e-05
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 820s 525ms/step - accuracy: 0.2122 - loss: 1.8282 - val_accuracy: 0.1496 - val_loss: 1.6721 - learning_rate: 1.0000e-05
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 812s 519ms/step - accuracy: 0.2105 - loss: 1.7757 - val_accuracy: 0.2743 - val_loss: 1.5968 - learning_rate: 1.0000e-05
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 0s 480ms/step - accuracy: 0.2042 - loss: 1.7298

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 808s 517ms/step - accuracy: 0.2042 - loss: 1.7298 - val_accuracy: 0.4046 - val_loss: 1.5695 - learning_rate: 1.0000e-05
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 808s 517ms/step - accuracy: 0.2067 - loss: 1.6880 - val_accuracy: 0.2284 - val_loss: 1.6139 - learning_rate: 1.0000e-05
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 0s 480ms/step - accuracy: 0.2092 - loss: 1.6540

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 808s 517ms/step - accuracy: 0.2092 - loss: 1.6540 - val_accuracy: 0.4062 - val_loss: 1.5755 - learning_rate: 1.0000e-05
Epoch 8/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 808s 517ms/step - accuracy: 0.2131 - loss: 1.6425 - val_accuracy: 0.1214 - val_loss: 1.6908 - learning_rate: 1.0000e-05
Epoch 9/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 806s 516ms/step - accuracy: 0.2078 - loss: 1.6347 - val_accuracy: 0.2292 - val_loss: 1.5906 - learning_rate: 1.0000e-05
Epoch 10/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 812s 519ms/step - accuracy: 0.2086 - loss: 1.6273 - val_accuracy: 0.4027 - val_loss: 1.5818 - learning_rate: 3.0000e-06


In [ ]:
test_gen.reset()
pred_probs = model.predict(test_gen)
y_pred = np.argmax(pred_probs, axis=1)
y_true = test_gen.classes


770/770 ━━━━━━━━━━━━━━━━━━━━ 3181s 4s/step


In [ ]:
print(classification_report(y_true, y_pred, digits=4))
print("Cohen Kappa:", cohen_kappa_score(y_true, y_pred))


              precision    recall  f1-score   support

           0     0.4471    0.9352    0.6050      5000
           1     0.0332    0.0054    0.0092      1862
           2     0.2857    0.0040    0.0079      2999
           3     0.3091    0.1350    0.1879       978
           4     0.1905    0.1398    0.1613      1466

    accuracy                         0.4092     12305
   macro avg     0.2531    0.2439    0.1943     12305
weighted avg     0.3036    0.4092    0.2833     12305

Cohen Kappa: 0.0723712182958619
